# FHIR CRUD using fhirpy

## Setup

In [1]:
!pip install fhirpy fhir.resources uuid

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import json
# from fhirpy import SyncFHIRClient
from fhirpy import AsyncFHIRClient
from fhir.resources.patient import Patient
from fhir.resources.humanname import HumanName
import uuid

In [3]:
server_url = 'http://localhost:52773/fhir/r4'
client = AsyncFHIRClient(
    url=server_url, 
    authorization='', 
    extra_headers={"Content-Type": "application/fhir+json;charset=utf-8"}
)

## Creating a patient

In [4]:
patient = Patient()
name = HumanName()
name.use = "official"
name.family = "familyname"
name.given = ["givenname1","givenname2"]
patient.name = [name]
patient

Patient(resource_type='Patient', fhir_comments=None, id=None, implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=None, contained=None, extension=None, modifierExtension=None, text=None, active=None, active__ext=None, address=None, birthDate=None, birthDate__ext=None, communication=None, contact=None, deceasedBoolean=None, deceasedBoolean__ext=None, deceasedDateTime=None, deceasedDateTime__ext=None, gender=None, gender__ext=None, generalPractitioner=None, identifier=None, link=None, managingOrganization=None, maritalStatus=None, multipleBirthBoolean=None, multipleBirthBoolean__ext=None, multipleBirthInteger=None, multipleBirthInteger__ext=None, name=[HumanName(resource_type='HumanName', fhir_comments=None, extension=None, id=None, family='familyname', family__ext=None, given=['givenname1', 'givenname2'], given__ext=None, period=None, prefix=None, prefix__ext=None, suffix=None, suffix__ext=None, text=None, text__ext=None, use='official', use__ext=None)],

In [5]:
# the fhirpy lib doen't return the ID of a resource 
# create by it - at least until the time of writing 
# of this notebook. A workaroud to manage that is to 
# define an ID
patient.id = str(uuid.uuid1())

In [6]:
patient = client.resource('Patient', **json.loads(patient.json()))
patient

<AsyncFHIRResource Patient/e7711428-8e4b-11ee-8238-0242ac180002>

In [7]:
await patient.save()

In [8]:
created_id = patient['id']

## Retrieving a patient

In [9]:
patient_resource = await client.reference('Patient', created_id).to_resource()
patient_resource.serialize()

{'resourceType': 'Patient',
 'id': 'e7711428-8e4b-11ee-8238-0242ac180002',
 'name': [{'use': 'official',
   'family': 'familyname',
   'given': ['givenname1', 'givenname2']}],
 'meta': {'lastUpdated': '2023-11-29T00:11:55Z', 'versionId': '1'}}

## Updating a patient

In [10]:
patient.name[0].family

'familyname'

In [11]:
patient.name[0].family = f'{patient.name[0].family} [updated]'
patient.name[0].family

'familyname [updated]'

In [12]:
await patient.save()

In [13]:
patient_resource = await client.reference('Patient', created_id).to_resource()
patient_resource.serialize()

{'resourceType': 'Patient',
 'id': 'e7711428-8e4b-11ee-8238-0242ac180002',
 'name': [{'use': 'official',
   'family': 'familyname [updated]',
   'given': ['givenname1', 'givenname2']}],
 'meta': {'lastUpdated': '2023-11-29T00:11:55Z', 'versionId': '2'}}

## Deleting a patient

In [14]:
await patient.delete()

In [15]:
try:
    patient_resource = await client.reference('Patient', created_id).to_resource()
except Exception as e:
    print(e)

{"resourceType":"OperationOutcome","issue":[{"severity":"error","code":"exception","diagnostics":"<HSFHIRErr>ResourceWasDeleted","details":{"text":"Resource was previously deleted (type='Patient', id='e7711428-8e4b-11ee-8238-0242ac180002', version='')."}}]}
